<a href="https://colab.research.google.com/github/shailavij/Process-Symbol-detection/blob/main/P%26ID_Symbol_detection_tutorial_12_07_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbol detection from P&ID 

 Tensorflow Object Detection to detect dynamic symbol from P&ID

### Install Trensorflow

In [ ]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

Make sure you have `pycocotools` installed

In [ ]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

# Configure Google drive to access Annotation image file and other data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/object_detect')

In [ ]:
pwd

### Imports Library dependencies

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:
os.chdir('/content/drive/MyDrive/object_detect/shailaj_pro_12july/data/annotation')
!ls

#Integration and Conversion of Images and Annotation Files into tf_record

In [ ]:
!python generate_tfrecord.py --csv_input=train_labels.csv  --output_path=train.record

In [ ]:
!python generate_tfrecord.py --csv_input=test_labels.csv  --output_path=test.record

In [ ]:
pip install tensorflow-addons

In [ ]:
pip install tensorflow-addons[tensorflow]

In [ ]:
import tensorflow_addons as tfa

# Train the model
Copy TensorFlow/models/research/object_detection/model_main_tf2.py script and paste it straight into our data folder. 

In [ ]:
!python '/content/models/research/object_detection/model_main_tf2.py'  --model_dir=/content/drive/MyDrive/object_detect/shailaj_pro_1 2july/Model/RCNN1 --pipeline_config_path=/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/ssd_mobilenet_v2_320x320/SSD_mobilenet_v2_pipeline.config

In [ ]:

%%capture
!pip install -q tf-nightly-gpu-2.0-preview
# Load the TensorBoard notebook extension
# %load_ext tensorboard.notebook # For older versions
%load_ext tensorboard

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
%load_ext tensorboard

In [ ]:
pwd

In [ ]:
# Configure path to 'train folder contains events. file'
%tensorboard --logdir '/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/RCNN1/train'

In [ ]:
# Configure path to 'train folder contains events. file'
%tensorboard --logdir '/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/RCNN1/eval'

# Export a trained Model

In [ ]:
!python '/content/models/research/object_detection/exporter_main_v2.py' --input_type image_tensor --pipeline_config_path=/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/ssd_mobilenet_v2_320x320/SSD_mobilenet_v2_pipeline.config --trained_checkpoint_dir=/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/RCNN1 --output_directory=/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/Output/output_inference_graph1.pb


# Evaluating the Model

In [ ]:
!python '/content/models/research/object_detection/model_main_tf2.py' --model_dir=/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/RCNN1 --pipeline_config_path=/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/ssd_mobilenet_v2_320x320/SSD_mobilenet_v2_pipeline.config  --checkpoint_dir=/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/RCNN1


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.
I have used an "SSD with Mobilenet" model from Tensorflow Model Zoo

## Loader

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

## Loading label map
Label maps map indices to category names

In [ ]:
# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
PATH_TO_LABELS= '/content/drive/MyDrive/object_detect/shailaj_pro_12july/data/annotation/label_map.pbtxt'

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
category_index

For the sake of simplicity we will test on 2 images:

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.JPG")))
#TEST_IMAGE_PATHS = sorted(list('/content/T12.jpg'))
TEST_IMAGE_PATHS

# Detection

Load an object detection model:

In [ ]:
model_name = '/content/drive/MyDrive/object_detect/shailaj_pro_12july/Model/Output/save_model1/saved_model'


In [ ]:
detection_model = tf.saved_model.load(model_name)

Check the model's input signature, it expects a batch of 3-color images of type uint8:

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

And returns several outputs:

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

In [ ]:
detection_model.signatures['serving_default'].output_shapes

Add a wrapper function to call the model, and cleanup the outputs:

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:

for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)
